# Exercise 4: Extracting Quantitative Measures from EEG spectra 

### This exercise makes use of data from the paper 

### Dissociable Decoding of Working Memory and Spatial Attention from EEG Oscillations and Sustained Potentials"

#### Bae, G. Y., & Luck, S. J. (2018). Dissociable Decoding of Spatial Attention and Working Memory from EEG Oscillations and Sustained Potentials. Journal of Neuroscience, 38(2), 409-422.

#### The paper is available in the readings folder.  

## This data set is potentially a very good data set to use for your final project as it is information rich. 

#### Having said that, I dislike many aspects of the paper.  

#### Load modules that we need 

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt
from hdf5storage import loadmat, savemat 
from scipy.fftpack import fft 
from scipy import signal



In [ ]:
data = loadmat('data/wm_orientation_506.mat')



In [ ]:
data.keys()


In [ ]:
#extract the variables from the dictionary 
eeg = data['eeg']
channel_names = data['channel_names']
angle = np.int16(data['angle'][0])
response = np.int16(data['response'][0])
samplingrate = int(data['samplingrate'][0])
time = np.int16(data['time'][0])
stimsamp = 375 #I looked at time to figure this out.  
ntrials = eeg.shape[0]
nchannels = eeg.shape[1]
nsamps = eeg.shape[2]

## README 

![Experiment](img/BaeLuckFigure1.jpg)

### In this experiment, participants are presented tear drops of different orientation for 200 ms.  

### After a delay period of 1300 ms, they rotate an arrow to match the orientation presented. 

### The data presented includes 1500 ms before the stimulus presented, 200 ms of stimulus presentation and 1300 ms of hold period. 

### The response period is not included. 

###  The variables contained here are 


*   `samplingrate` - number of samples of EEG in per second
*   `eeg` - eegdata of the experiment.  of dimensions, ntrials x nchannels x ntimepoints.  The EEG is provided in units of volts. 
*   `time` - the time relative to *stimulus onset* in each EEG observation.
*   `channel_names` - the name of the EEG channels indicating where it is located.    
*   `angle` - the angle of the teardrop stimulus  
*   `response` - the angle reported by the subject. 



## Problem #1


#### In the tutorial example. we compared the response when the task was to remember an angle of 0 degrees to the response when the angle was 180 degrees using 1 second after the stimulus presentation

`poststim = range(375,625,1)`

#### However, during this interval, the stimulus is presented on the screen for the first 200 ms (50 samples at a sampling rate of 250), so we may just be differentiating the response to the stimulus rather than working memory. The most task relevant interval would be the period just prior to the response, when they are holding the item in memory and preparing to make a response. 

#### a. Divide the 1.5 sec interval (375 samples) after stimulus presentation into 3 intervals of length 0.5 second (125 samples) and compute the average amplitude spectrum for each angle from 0 to 360 for each of the 3 intervals.  So there are 3 intervals 

* 0 -0.5s, samples 375-500
* 0.5 -1s, samples 500-625
* 1 -1.5s, samples 625- 750

#### b. For visualization, we will focus on PO7 and PO8 over the left and right hemispheres respectively, and the angles 0 and 180 which are pointing to the left and right.   For each of the three intervals, make a separate graph for PO7 and PO8 plotting the amplitude spectrum of the EEG for stimuli in memory at angle 0 and angle 180.  

#### c. Extract the amplitude values at 10 Hz for all 27 channels in the interval using the amplitude spectrum computed from the interval from 1-1.5s.  Following the tutorial make a plot of the amplitude at each channel at 10 Hz.   